# Project 3: Decision Tree

## Part A: Iris dataset

### 1. Import libraries and dataset

In [8]:
# import libraries
import numpy as np
import pandas as pd
import csv
import math
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

In [9]:
# import data
data = pd.read_csv("iris.csv", header=None)
data.head(5)

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


### 2. Define necessary functions

#### a) Node

In [10]:
class Node():
    
    # constructor
    def __init__(self, feature_num=None, threshold=None, gain=None, left=None, right=None, leaf_val=None):
        
        # stores with which feature the node makes the decisions.
        self.feature_num = feature_num 
        
        # stores the threshold value for the node makeing the decisions.
        self.threshold = threshold
        
        # the information gain obtained
        self.gain = gain
        
        # the left node
        self.left = left
        
        # the right node
        self.right = right
        
        # the value of the node when it is the leaf node. 
        self.leaf_val = leaf_val
        

#### b) Tree

In [11]:
class Tree():
    
    # 1. constructor:
    def __init__(self, min_row_num):
        
        # the root node
        self.root = None
        
        # the minimum row requirement
        self.min_row_num=min_row_num
        
   
    # 2. the function that makes the entire tree (we use recursion method)
    def make_tree(self, dataset):
        
        # split the dataset to X and y
        X = dataset[:,:-1]
        y = dataset[:,-1]
        
        # number of rows and columns of X each nodes is filtering.
        row, col = np.shape(X)
        
        # check if the number of instances reached the minimum requirement. 
        if row > self.min_row_num:
            # divide the data.
            divider_info = self.divider(dataset)
            # check if the information gain is greater than zero. 
            if divider_info["gain"] > 0:
                # run the recursion for both left node and right node.
                l = self.make_tree(divider_info["left"])
                r = self.make_tree(divider_info["right"])
                # recursively stack up the nodes in the tree
                return Node(divider_info["feature_num"], divider_info["threshold"], divider_info["gain"], l, r)
            
        # compute the value of the leaf node. The value should be the one with the highest frequencty in y_list
        y_list = list(y)
        result = max(set(y_list), key = y_list.count)
        return Node(leaf_val = result)
    
    
    # 3. The function that divides the data maximizing the information gain:
    def divider(self, dataset):
        
        # a dictionary that stores all the information about the splitting.
        divider_info = {}
        # a variable "max_gain" for discovering the best split
        max_gain = -10000
        
        # number of rows and columns of X.
        row, col = np.shape(dataset)
        col = col-1
        
        # iterate through all the possible threshold values (inner loop) in all the possible features (outer loop)
        for c in range(col): 
            # in order to reduce repetitive calculation, we use "np.unique".
            column = dataset[:,c]
            thresholds = np.unique(column)
            
            for t in thresholds:
                left = dataset[dataset[:,c]<t]
                right = dataset[dataset[:,c]>t]

                # check if there is anything trials filtering all the data to one side.
                if len(left) > 0 and len(right)>0:
                    
                    # compute the information gain
                    w_left = len(left)/len(dataset)
                    w_right = len(right)/len(dataset)
                    
                    curr_gain = self.get_entropy(dataset[:,-1])-w_left*self.get_entropy(left[:,-1])-w_right*self.get_entropy(right[:,-1])

                    # update the split info if curr_gain > max_gain
                    if curr_gain > max_gain:
                        # update the max gain
                        max_gain = curr_gain
                        # update the split_info
                        divider_info["feature_num"] = c
                        divider_info["threshold"] = t
                        divider_info["left"] = left
                        divider_info["right"] = right
                        divider_info["gain"] = curr_gain
        
        return divider_info
                    
                    
    # 4. The function to compute entropy
    def get_entropy(self,y):
        
        # obtain all the unique categories in y
        categories = np.unique(y)
        entropy = 0
        
        # compute the entropy
        for category in categories:
            probability = len(y[y==category])/len(y)
            entropy += (-1) * probability * math.log(probability,2)
            
        return entropy
       
        
    # 5. the recursive function to compute prediction of a single instance
    def single_predict (self, x, node):
        
        # check if it reached the leaf
        if node.leaf_val != None:
            return node.leaf_val
        
        # obtain the feature value of the instance
        feature_value = x[node.feature_num]
        
        # check left or right, and make recursive calls
        if feature_value < node.threshold:
            return self.single_predict(x,node.left)
        else:
            return self.single_predict(x,node.right)            

    # 6. the function that predicts the entire X.
    def get_predict(self, X):
        result = []
        for x in X:
            result.append(self.single_predict(x,self.root))
        
        return result
        
        

### 3. Compute the accuracy and the std with k-fold cross validation.

In [13]:
min_percent_list = [5,10,15,20]
dataset = data.to_numpy()
total_row_num = len(dataset)

for min_percent in min_percent_list:
    
    accuracy_list = []
    
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    
    # obtain the instance limit by multiplying the percentage and the total_instance
    min_row_num = math.ceil(total_row_num * min_percent / 100)
    
    print("When minimum percent is "+str(min_percent)+":")
    trial =1
    for train, test in kf.split(dataset):
        dataset_train = dataset[train]
        dataset_test = dataset[test]
        
        # train the decision tree with corresponding min_percent value
        tree = Tree(min_row_num)
        tree.root=tree.make_tree(dataset_train)

        # get the prediction result
        y_prediction = tree.get_predict(dataset_test[:,:-1])
        accuracy = accuracy_score(dataset_test[:,-1],y_prediction)
        accuracy_list.append(accuracy)
        
        print("trial "+str(trial)+": "+str(accuracy))
        trial += 1
    print("The mean of the accuracy is: "+str(np.mean(accuracy_list))) 
    print("The standard deviation of the accuracy is: "+str(np.std(accuracy_list)))
    print("")


When minimum percent is 5:
trial 1: 1.0
trial 2: 1.0
trial 3: 1.0
trial 4: 0.9333333333333333
trial 5: 1.0
trial 6: 0.8666666666666667
trial 7: 0.8666666666666667
trial 8: 1.0
trial 9: 0.9333333333333333
trial 10: 0.8666666666666667
The mean of the accuracy is: 0.9466666666666667
The standard deviation of the accuracy is: 0.0581186525805423

When minimum percent is 10:
trial 1: 1.0
trial 2: 1.0
trial 3: 1.0
trial 4: 0.9333333333333333
trial 5: 1.0
trial 6: 0.8666666666666667
trial 7: 0.8666666666666667
trial 8: 1.0
trial 9: 0.9333333333333333
trial 10: 0.8666666666666667
The mean of the accuracy is: 0.9466666666666667
The standard deviation of the accuracy is: 0.0581186525805423

When minimum percent is 15:
trial 1: 1.0
trial 2: 1.0
trial 3: 1.0
trial 4: 0.9333333333333333
trial 5: 1.0
trial 6: 0.8666666666666667
trial 7: 0.8666666666666667
trial 8: 1.0
trial 9: 0.9333333333333333
trial 10: 0.8666666666666667
The mean of the accuracy is: 0.9466666666666667
The standard deviation of the

## Part B: Spambase dataset

### 1. import dataset

In [14]:
# import data
data = pd.read_csv("spambase.csv", header=None)
data.head(5)

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


### 2. Compute the accuracy and the std with k-fold cross validation.

In [15]:
min_percent_list = [5,10,15,20,25]
dataset = data.to_numpy()
total_row_num = len(dataset)

for min_percent in min_percent_list:
    
    accuracy_list = []
    
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    
    # obtain the instance limit by multiplying the percentage and the total_instance
    min_row_num = math.ceil(total_row_num * min_percent / 100)
    
    print("When minimum percent is "+str(min_percent)+":")
    trial =1
    for train, test in kf.split(dataset):
        dataset_train = dataset[train]
        dataset_test = dataset[test]
        
        # train the decision tree with corresponding min_percent value
        tree = Tree(min_row_num)
        tree.root=tree.make_tree(dataset_train)

        # get the prediction result
        y_prediction = tree.get_predict(dataset_test[:,:-1])
        accuracy = accuracy_score(dataset_test[:,-1],y_prediction)
        accuracy_list.append(accuracy)
        
        print("trial "+str(trial)+": "+str(accuracy))
        trial += 1
    print("The mean of the accuracy is: "+str(np.mean(accuracy_list))) 
    print("The standard deviation of the accuracy is: "+str(np.std(accuracy_list)))
    print("")


When minimum percent is 5:
trial 1: 0.89587852494577
trial 2: 0.8913043478260869
trial 3: 0.8891304347826087
trial 4: 0.9065217391304348
trial 5: 0.8673913043478261
trial 6: 0.9043478260869565
trial 7: 0.8978260869565218
trial 8: 0.9043478260869565
trial 9: 0.9
trial 10: 0.9195652173913044
The mean of the accuracy is: 0.8976313307554464
The standard deviation of the accuracy is: 0.012983964563038913

When minimum percent is 10:
trial 1: 0.8763557483731019
trial 2: 0.8869565217391304
trial 3: 0.8891304347826087
trial 4: 0.8913043478260869
trial 5: 0.8739130434782608
trial 6: 0.908695652173913
trial 7: 0.8739130434782608
trial 8: 0.9065217391304348
trial 9: 0.8978260869565218
trial 10: 0.9043478260869565
The mean of the accuracy is: 0.8908964444025276
The standard deviation of the accuracy is: 0.012637377372193576

When minimum percent is 15:
trial 1: 0.8394793926247288
trial 2: 0.8543478260869565
trial 3: 0.8956521739130435
trial 4: 0.8543478260869565
trial 5: 0.8347826086956521
trial 6